In [80]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Flatten , Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import plot_model #모델 시각화 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, TensorBoard


tf.config.run_functions_eagerly(False)



## 데이터 불러오기
- 앞서 크롤링하여 엑셀에 저장한 파일을 불러온다.

In [81]:
data_df = pd.read_excel('baseball_data.xlsx')
target_df = pd.read_excel('label.xlsx')
try:
    del data_df['Unnamed: 0']
    del target_df['Unnamed: 0']
except:
    pass


## 훈련세트, 검증세트, 테스트세트 분리
- 8:2비율로 훈련세트, 테스트 세트를 분리하고 또 다시 8:2로 훈련세트, 검증세트를 분리한다.
- 스코어를 예측하는 회귀모델을 위해 훈련세트, 검증세트, 테스트세트를 A팀과 B팀으로 분리한다.
- 레이블도 A팀 스코어 레이블, B팀 스코어 레이블, 승패 레이블로 분리한다.

In [82]:
#훈련세트, 검증세트, 테스트세트 분리 
X_full_set, X_test=  train_test_split(data_df, test_size = 0.2, random_state = 7)
y_full_set, y_test = train_test_split(target_df, test_size = 0.2, random_state = 7)
x_train, x_val, y_train, y_val = train_test_split(X_full_set,y_full_set, test_size=0.2, random_state=7)

#A B 나누기
A_train = x_train.iloc[:,:14]
B_train = x_train.iloc[:,14:]
A_valid = x_val.iloc[:,:14]   
B_valid = x_val.iloc[:,14:]
A_test = X_test.iloc[:,:14]
B_test = X_test.iloc[:,14:]

winlose_target_train = pd.DataFrame(y_train['승패'])
A_score_train = pd.DataFrame(y_train['A팀_스코어'])
B_score_train = pd.DataFrame(y_train['B팀_스코어'])

winlose_target_valid = pd.DataFrame(y_val['승패'])
A_score_valid = pd.DataFrame(y_val['A팀_스코어'])
B_score_valid = pd.DataFrame(y_val['B팀_스코어'])

winlose_target_test = pd.DataFrame(y_test['승패'])
A_score_test = pd.DataFrame(y_test['A팀_스코어'])
B_score_test = pd.DataFrame(y_test['B팀_스코어'])



## 데이터 전처리
- min-max 스케일링(정규화) 와 표준화 둘 중 더 좋은걸로 선택
- 전처리 하지 않는걸로...

In [83]:
#데이터 정규화
#scaler = StandardScaler() 
#mms = MinMaxScaler()

#x_train = scaler.fit_transform(x_train)
#x_val=scaler.fit_transform(x_val)
#X_test=scaler.fit_transform(X_test)
#x_val=scaler.fit_transform(x_val)

In [84]:
#A_train= scaler.fit_transform(A_train)
#B_train= scaler.fit_transform(B_train)

#A_valid=scaler.fit_transform(A_valid)
#B_valid=scaler.fit_transform(B_valid)

#A_test=scaler.transform(A_test)
#B_test=scaler.transform(B_test)

In [85]:
#차원축소
#pca = PCA(n_components=7)
#X_train = pca.fit_transform(X_train)
#X_valid = pca.fit_transform(X_valid)


## 모델 만들기
- 모델1(점수예측)은 회귀모델이기 때문에 출력층에 활성화함수를 지정하지 않았다.
- 모델2(승패예측)은 이진분류모델이기 때문에 출력층에 시그모이드 함수를 지정했다.

In [86]:
#모델1 만들기
input = Input(shape=(14,),name='input')
hidden1 = Dense(400,activation='relu',name='hidden1')(input)
hidden2 = Dense(200,activation='relu',name='hidden2')(hidden1)
hidden3= Dense(50,activation='relu',name='hidden3')(hidden2)
hidden4 = Dense(30,activation='relu',name='hidden_4')(hidden3)
output = Dense(1,name='output')(hidden4)


model_1 = Model(inputs=input,outputs=output)

#모델2 만들기
input= Input(shape=(28,),name='input')
hidden_1 = Dense(400,activation='relu',name='hidden_1')(input)
hidden_2 = Dense(200,activation='relu',name='hidden_2')(hidden_1)
hidden_3 = Dense(50,activation='relu',name='hidden_3')(hidden_2)
hidden_4 = Dense(30,activation='relu',name='hidden_4')(hidden_3)
output = Dense(1,activation='sigmoid',name='output')(hidden_4)

model_2 = Model(inputs=input,outputs=output)

## 모델컴파일
- 회귀문제 : MSE(loss) , MAE(eval)
- 이진분류문제 : binary cross entropy(loss) , accuracy(eval)

In [87]:
model_1.compile(loss='mse',
              optimizer=Adam(learning_rate=1e-2),metrics=['mae'])

model_2.compile(loss='binary_crossentropy', optimizer='Adam',metrics=['accuracy'])

## 모델학습

In [88]:
#A데이터로 model_1 학습
history = model_1.fit(A_train,
                    A_score_train,
                    epochs=30, validation_data=(A_valid,A_score_valid))

Epoch 1/30
39/39 [==============================] - 1s 8ms/step - loss: 19.7558 - mae: 3.1443 - val_loss: 4.9938 - val_mae: 1.8052
Epoch 2/30
39/39 [==============================] - 0s 4ms/step - loss: 2.9752 - mae: 1.3745 - val_loss: 2.6805 - val_mae: 1.2733
Epoch 3/30
39/39 [==============================] - 0s 6ms/step - loss: 3.0689 - mae: 1.3653 - val_loss: 2.4287 - val_mae: 1.2383
Epoch 4/30
39/39 [==============================] - 0s 5ms/step - loss: 2.7197 - mae: 1.2807 - val_loss: 2.8811 - val_mae: 1.3553
Epoch 5/30
39/39 [==============================] - 0s 5ms/step - loss: 2.4326 - mae: 1.2167 - val_loss: 2.8701 - val_mae: 1.3049
Epoch 6/30
39/39 [==============================] - 0s 3ms/step - loss: 2.4816 - mae: 1.2251 - val_loss: 2.8986 - val_mae: 1.3662
Epoch 7/30
39/39 [==============================] - 0s 4ms/step - loss: 2.6320 - mae: 1.2779 - val_loss: 2.2434 - val_mae: 1.1931
Epoch 8/30
39/39 [==============================] - 0s 3ms/step - loss: 2.7202 - mae: 1.2

In [89]:
#B데이터로 model_1 학습 
history = model_1.fit(B_train,
                    B_score_train,
                    epochs=30, validation_data=(B_valid,B_score_valid))

Epoch 1/30
39/39 [==============================] - 0s 4ms/step - loss: 2.5149 - mae: 1.2379 - val_loss: 2.3799 - val_mae: 1.2215
Epoch 2/30
39/39 [==============================] - 0s 3ms/step - loss: 2.9657 - mae: 1.3106 - val_loss: 2.5986 - val_mae: 1.2920
Epoch 3/30
39/39 [==============================] - 0s 3ms/step - loss: 2.5237 - mae: 1.2456 - val_loss: 2.3571 - val_mae: 1.1959
Epoch 4/30
39/39 [==============================] - 0s 3ms/step - loss: 2.4826 - mae: 1.2259 - val_loss: 2.6486 - val_mae: 1.2891
Epoch 5/30
39/39 [==============================] - 0s 4ms/step - loss: 2.7389 - mae: 1.3051 - val_loss: 4.6580 - val_mae: 1.7250
Epoch 6/30
39/39 [==============================] - 0s 3ms/step - loss: 2.8181 - mae: 1.2947 - val_loss: 3.4313 - val_mae: 1.4738
Epoch 7/30
39/39 [==============================] - 0s 4ms/step - loss: 2.5446 - mae: 1.2403 - val_loss: 2.5954 - val_mae: 1.2775
Epoch 8/30
39/39 [==============================] - 0s 3ms/step - loss: 2.4771 - mae: 1.22

In [90]:
#X데이터로 model_2 학습
history = model_2.fit(x_train,winlose_target_train, epochs=30, validation_data=(x_val,winlose_target_valid))

Epoch 1/30
39/39 [==============================] - 1s 9ms/step - loss: 0.6895 - accuracy: 0.6786 - val_loss: 0.4689 - val_accuracy: 0.7630
Epoch 2/30
39/39 [==============================] - 0s 4ms/step - loss: 0.4171 - accuracy: 0.8084 - val_loss: 0.3653 - val_accuracy: 0.8312
Epoch 3/30
39/39 [==============================] - 0s 8ms/step - loss: 0.3441 - accuracy: 0.8450 - val_loss: 0.3099 - val_accuracy: 0.8734
Epoch 4/30
39/39 [==============================] - 0s 6ms/step - loss: 0.3202 - accuracy: 0.8547 - val_loss: 0.3114 - val_accuracy: 0.8571
Epoch 5/30
39/39 [==============================] - 0s 4ms/step - loss: 0.3222 - accuracy: 0.8620 - val_loss: 0.2698 - val_accuracy: 0.8831
Epoch 6/30
39/39 [==============================] - 0s 5ms/step - loss: 0.3573 - accuracy: 0.8401 - val_loss: 0.3350 - val_accuracy: 0.8539
Epoch 7/30
39/39 [==============================] - 0s 5ms/step - loss: 0.3253 - accuracy: 0.8506 - val_loss: 0.2786 - val_accuracy: 0.8734
Epoch 8/30
39/39 [==

## 모델 평가

In [91]:
model_1.evaluate(A_test,A_score_test)
model_2.evaluate(X_test,winlose_target_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.2863 - accuracy: 0.9091


[0.28634652495384216, 0.9090909361839294]

## 모델 저장

In [92]:
#모델 저장
model_1.save('score_predict_model.h5')
model_2.save('win_lose_predict_model.h5')


## 예측
- 학습하지 않은 22년 10월 8일 3경기의 데이터를 예측하고자 한다.
- 단 이때 22년 10월 8일의 경기가 진행되지 않음을 가정하고 예측을 진행한다. 스코어예측이나 점수예측은 주로 경기 전에 진행되는
  특성을 따른 것.
- 때문에 모델에 input으로 들어가는 데이터는 각 구단의 직전 3경기의 평균값을 입력하고 실제 22년 10월 8일의 데이터와 비교한다.
- 선발투수는 경기전 확정된 선발투수의 데이터로 입력값을 넣는다(직전 3경기 평균값X 당일 데이터O)
|22-10-8|A구단 점수|B구단 점수|
|------|----------|----------|
|SSG(A) : 삼성(B)|1|6|
|키움 : 두산|5|1|
|한화 : NC|5|6|


In [93]:
#예측1 SSG:삼성
SSG =[36.3,33.6,6.6,1,2.3,0,9.6,10,7.6,0.196,0.153,18,2.49,1.16]
samsung = [41.6,38.6,12.6,0.3,2.3,0,8,8.6,9.3,0.325,0.252,19,6.00,1.52]
dusan = [34.6,32.6,9.3,1,1.3,0,6.3,8,9,0.282,0.213,11.6,3.60,1.33]
kiwum = [50,44,12,1,5.3,0.6,8.6,11.6,11.6,0.268,0.213,21.6,2.11,0.95]
hanwha = [42.6,34.6,9,0,5.6,1,6.3,10.6,9,0.258,0.201,24.6,4.75,1.42]
NC = [34.3,30.6,6.6,0.3,3,2,8.6,9,7,0.216,0.198,11.6,10.64,1.91]

score_SSG_samsung =model_1.predict([SSG,samsung])
score_dusan_kiwum =model_1.predict([dusan,kiwum])
score_hanwha_NC =model_1.predict([hanwha,NC])
winlose_SSG_samsung= model_2.predict([[36.3,33.6,6.6,1,2.3,0,9.6,10,7.6,0.196,0.153,18,2.49,1.16,41.6,38.6,12.6,0.3,2.3,0,8,8.6,9.3,0.325,0.252,19,6.00,1.52]])
winlose_dusan_kiwum = model_2.predict([[34.6,32.6,9.3,1,1.3,0,6.3,8,9,0.282,0.213,11.6,3.60,1.33,50,44,12,1,5.3,0.6,8.6,11.6,11.6,0.268,0.213,21.6,2.11,0.95]])
winlose_hanwha_NC = model_2.predict([[42.6,34.6,9,0,5.6,1,6.3,10.6,9,0.258,0.201,24.6,4.75,1.42,34.3,30.6,6.6,0.3,3,2,8.6,9,7,0.216,0.198,11.6,10.64,1.91]])
print(f"SSG 예측점수:{score_SSG_samsung[0]}")
print(f"삼성 예측점수:{score_SSG_samsung[1]}")
print(f"두산 예측점수:{score_dusan_kiwum[0]}")
print(f"키움 예측점수:{score_dusan_kiwum[1]}")
print(f"한화 예측점수:{score_hanwha_NC[0]}")
print(f"NC 예측점수:{score_hanwha_NC[1]}")
if winlose_SSG_samsung < 0.5:
    print("SSG-승, 삼성-패")
else:
    print("삼성-승, SSG-패")

if winlose_dusan_kiwum < 0.5:
    print("두산-승, 키움-패")
else:
    print("키움-승, 두산-패")
    
if winlose_dusan_kiwum < 0.5:
    print("한화-승, NC-패")
else:
    print("NC-승, 한화-패")
print("=======================================================")
print(winlose_SSG_samsung)
print(winlose_dusan_kiwum)
print(winlose_hanwha_NC)


1/1 [==============================] - 0s 72ms/step
SSG 예측점수:[1.6119151]
삼성 예측점수:[5.1620493]
두산 예측점수:[3.860777]
키움 예측점수:[6.276722]
한화 예측점수:[4.0603266]
NC 예측점수:[3.9604685]
삼성-승, SSG-패
두산-승, 키움-패
한화-승, NC-패
[[0.9621753]]
[[0.09099566]]
[[0.6634263]]
